Get links for search

In [6]:
import requests
import time
from bs4 import BeautifulSoup

# Set the Amazon search URL and search term
search_term = 'headphone'
search_url = f"https://www.amazon.in/s?k={search_term}"



# Create a session to manage cookies and maintain state
session = requests.Session()

# Set a User-Agent header to mimic a common web browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Send an initial request to the search URL
response = session.get(search_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the search page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links with the specified class
    links_with_class = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')

    # Extract the href attribute from each link and store it in a list
    link_list = [link.get('href') for link in links_with_class]

    # Append "https://www.amazon.in" to links beginning with "/"
    for i in range(len(link_list)):
        if link_list[i].startswith('/'):
            link_list[i] = 'https://www.amazon.in' + link_list[i]

    # Create a filename based on the search term
    filename = f'{search_term}_links.txt'

    # Save the list of modified links to a text file with the search term as the filename
    with open(filename, 'w') as file:
        for link in link_list:
            file.write(link + '\n')

    # Print the list of modified links
    for link in link_list:
        print(link)

    # Add a delay between requests to avoid overloading the server
    time.sleep(3)
else:
    print("Failed to retrieve the Amazon search page. Status code:", response.status_code)


Failed to retrieve the Amazon search page. Status code: 503


Scrape

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

# Function to extract Product title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        title_value = title.text
        title_string = title_value.strip()
    except Exception:
        title_string = ""
    return title_string

# Function to extract Product Prices
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'}).string.strip()
    except Exception:
        try:
            price = soup.find("span", attrs={'id': 'priceblock_dealprice'}).string.strip()
        except:
            price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
    except Exception:
        try:
            rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
        except:
            rating = ""
    return rating

# Function to extract the number of reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.split()
    except Exception:
        review_count = ""
    return review_count

# Function to extract the Product description
def get_product_description(soup):
    try:
        description = soup.find("ul", class_="a-unordered-list a-vertical a-spacing-mini")
        if description:
            description_text = " ".join([item.text.strip() for item in description.find_all("span", class_="a-list-item")])
            return description_text
    except Exception:
        return ""

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip()
    except Exception:
        available = "Not Available"
    return available

# Function to extract product reviews
def get_reviews(soup):
    reviews = []
    review_elements = soup.find_all("div", class_="a-row a-spacing-small review-data")
    
    for element in review_elements[:10]:  # Extract the first 10 reviews
        review_text = element.find("span", class_="a-size-base review-text").text.strip()
        reviews.append(review_text)
    
    return reviews

# List of URLs to scrape
def read_urls_from_file(filename):
    with open(filename, 'r') as file:
        urls = [line.strip() for line in file]
    return urls

filename = 'health_links.txt'  # Replace with your filename
urls = read_urls_from_file(filename)

d = {"title": [], "price": [], "rating": [], "reviews_count": [], "availability": [], "description": [], "reviews": []}

for url in urls:
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        }
        # Create a session to manage cookies and maintain state
        session = requests.Session()
        # Use the session for making requests
        response = session.get(url, headers=headers)
        if response.status_code == 200:
            
            soup = BeautifulSoup(response.text, 'html.parser')
            d['title'].append(get_title(soup))
            d['price'].append(get_price(soup))
            d['rating'].append(get_rating(soup))
            d['reviews'].append(get_reviews(soup))
            d['availability'].append(get_availability(soup))
            d['description'].append(get_product_description(soup))
            print(get_title(soup))
        else:
            print(f"Failed to retrieve the Amazon page: {url}")
        # Ensure all lists have the same length by appending empty strings if needed
        for key in d:
            if len(d[key]) < len(d['title']):
                d[key].append("")
    except Exception as e:
        print(f"An error occurred: {e}")

# Continue with the rest of your code for data processing and CSV export

# Read the existing CSV file, if it exists
existing_df = pd.read_csv("amazon_data.csv")

# Concatenate the new data with the existing data
combined_df = pd.concat([existing_df, pd.DataFrame.from_dict(d)])

# Remove duplicate rows based on the 'title' column
combined_df.drop_duplicates(subset=['title'], keep='last', inplace=True)

# Save the combined DataFrame to the CSV file
combined_df.to_csv("amazon_data.csv", header=True, index=False)





Health OK - Bottle of 200 ml Syrup
HealthKart HK Vitals Fish Oil Capsule For Men And Women, 60 Omega 3 Fish Oil Capsules (1000mg Omega 3 with 180 mg EPA & 120 mg DHA), for Brain, Heart, Eyes, and Joints Health,
Himalaya Ashvagandha - 60 Tablets | Stress Relief | Rejuvenates Mind & Body | Improves Strength & Energy | General Wellness
Centrum Men Tablet, World's No.1 Multivitamin with Grape seed extract, Vitamin C & 21 other nutrients for Overall Health, Strong Muscles & Immunity (Veg) 30s
ALTON SHR20855 ABS Health Faucet with SS-304 Grade 1.25 Meter Flexible Hose Pipe and Wall Hook, White (Jet Spray for Toilet), Polished
Manna Health Mix 1kg | Multi Millet Health Drink Mix Powder for Kids | Millets, Nuts, Cereals & Pulses | Sathu maavu | Porridge Mix
Manna Health Mix 600g | Health and Nutrition Drink | Millets, Nuts, Cereals & Pulses | Sathu maavu | Porridge Mix
Dabur Chyawanprash - 950g | 3X Immunity Action | With 40+ Ayurvedic Herbs | Helps Build Strength & Stamina | Builds Overal